#    The Spread of COVID-19 in US 

In [234]:
# import the necessary libraries:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import datetime

In [235]:
# load the data:
current_states = pd.read_json("https://covidtracking.com/api/states")
states_daily   = pd.read_json("https://covidtracking.com/api/states/daily")

In [236]:
df =current_states.groupby(["state","dateChecked"])["positive","negative","death","hospitalized"].sum().reset_index()
df.head()

/Users/karimaidrissi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,state,dateChecked,positive,negative,death,hospitalized
0,AK,2020-03-28T12:42:00Z,85,2836.0,2.0,5.0
1,AL,2020-03-28T14:18:00Z,644,4184.0,3.0,0.0
2,AR,2020-03-28T13:51:00Z,388,2922.0,3.0,48.0
3,AS,2020-03-28T02:48:00Z,0,0.0,0.0,0.0
4,AZ,2020-03-28T13:09:00Z,738,7455.0,13.0,0.0


In [237]:
# Map showing cumulative of positive cases over time 
df =current_states.groupby(["state","dateChecked"])["positive","negative","death","hospitalized"].sum().reset_index()

fig = px.choropleth(locations =df['state'],
                   color =df['positive'],
                   locationmode="USA-states",
                   scope="usa",
                   animation_frame = df["dateChecked"],
                   color_continuous_scale="Viridis")
layout = go.Layout(
    title=go.layout.Title(text="Count positive cases of COVID-19 in US states",
                         x=0.5),
    font=dict(size=10),
)
fig.layout.template="plotly_dark"
fig.update_layout(layout)
fig.show()

/Users/karimaidrissi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [238]:
# the highest states in US:
positive = states_daily.groupby(["state"])['positiveIncrease'].sum().reset_index().sort_values(by='positiveIncrease', ascending=False).head(5)
fig = go.Figure(data=[go.Bar(name="positiveIncrease", x=positive["state"], y=positive["positiveIncrease"],marker_color ="orange"),])
fig.update_layout(barmode="group", title_text="Confirmed Cases in Top 5 states")
fig.layout.template="plotly_dark"
fig.show()

In [239]:
# fig shows number of death cases over time by each state:
df =states_daily.groupby(["state","dateChecked"])["positive","negative","death","hospitalized"].sum().reset_index()
#convert the date to a datetime object
df["dateChecked"] = pd.to_datetime(df["dateChecked"], format ="%Y-%m-%dT%H:%M:%SZ").dt.date
df = df.sort_values(by="dateChecked").reset_index(drop=True)
start_date = datetime.date(2020,3,1)
df = df[df["dateChecked"]>=start_date]
df["dateChecked"] = df["dateChecked"].astype(str)
fig = px.scatter(df, x="positive", y="death",animation_frame="dateChecked", animation_group="state",
                size="positive", color="state", hover_name="state",
                log_x=False, size_max=55, range_x=[0,30000], range_y=[-20,500])

layout = go.Layout(title=go.layout.Title(
                    text="the number of confirmed and death cases over time in USA", x=0.5),
                    font=dict(size=14),
                    xaxis_title="Total number of positive cases",
                    yaxis_title="Total number of death cases")
fig.update_layout(layout)
fig.layout.template="plotly_dark"
fig.show()

/Users/karimaidrissi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [240]:
# latest condensed data:
df1 = states_daily.groupby(["dateChecked"])["positive","negative","death","hospitalized"].sum().reset_index()
df1 = df1[df1["dateChecked"]==max(df1["dateChecked"])]
df1

/Users/karimaidrissi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,dateChecked,positive,negative,death,hospitalized
23,2020-03-27T20:00:00Z,99447.0,527220.0,1530.0,13717.0


In [241]:
# Loading latest condensed data using treemap:
total = df1.melt(id_vars="dateChecked", value_vars=['positive', 'negative', 'death','hospitalized'])
fig2 = px.treemap(total, path=["variable"], values="value", height=400, width=600,
                 color_discrete_sequence=['blue','orange'  ,'red',"green" ])
fig2.show()

In [242]:
# positive cases vs hospitalized over time:
positive = states_daily.groupby("dateChecked")['positive',"hospitalized"].sum().reset_index()

fig = go.Figure()
fig.add_trace(go.Scatter(x=positive["dateChecked"], y=positive["positive"],
                        mode="lines+markers",
                        name="Positive"))
fig.add_trace(go.Scatter(x=positive["dateChecked"], y=positive["hospitalized"],
                        mode="lines+markers",
                        name="Hospitalized"))
fig.update_layout(barmode="stack",title_text="total Confirmed cases vs hospitalized")
fig.layout.template="plotly_dark"
fig.show()

/Users/karimaidrissi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [243]:
# Death Cases by each states over time:
fig = px.line(states_daily.groupby(["dateChecked",'state']).agg({'death':"sum"}).reset_index(),x = "dateChecked",y = 'death',color = "state")
fig.update_layout(title_text = 'Death Cases by eash States in US')
fig.layout.template ='plotly_dark'
fig.show()